In [1]:
import openai;
import time;
import re;
import requests;
import os;
import random;
import pandas as pd;
import numpy as np;
from dotenv import load_dotenv;
import csv;
import time;
from openai import AzureOpenAI;

# loading of environment variables, where the api keys should be stores
load_dotenv();
openai.api_type = "azure";
openai.api_version = "2023-05-15" ;
openai.api_base = os.getenv("ENDPOINT");
openai.api_key = os.getenv("OPENAI_KEY");

# object to store api keys
client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint=openai.api_base,
);

In [2]:
# function that takes in the api keys "client" and a prompt "question"
def query_store(client, question):
    model_name = "gpt-4-32k";  
    time_start = time.time();
    
    # Create the prompt
    prompt = f"Q: {question} A:";
    
    # Create a chat completion request
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "assistant",
                "content": prompt,
            },
        ],
    );
    
    # Extract the content of the response
    answer = response.choices[0].message.content;
    time_elapsed = time.time() - time_start;

    # Prepare a dictionary with the information
    result = {
        'model_name': model_name,
        'runtime_seconds': time_elapsed,
        'question': question,
        'answer': answer
    };
    
    return result;

In [ ]:
template = "There was a GENDER patient of age AGE who was hospitalized for CONDITION for X days. During their hospitalization, they DID/DIDNOT discuss their goals of care. For each day, give me a summary detailing age, sex, and hospital course. ADDENDUM"

In [ ]:
def generate_question(row, template):
    gender = "male" if row['Sex'] == 1 else "female"
    age = row['Age']
    condition = "Cancer" if row['Cancer'] == 1 else "Dementia"
    days = row['Days']
    did_or_didnot = "did" if row['GOCD'] == 1 else "did not"
    addendum = f"The goals of care discussion took place on day {row['DayofGOCD']}." if row['GOCD'] == 1 else ""

    summary = template.replace("GENDER", gender) \
                      .replace("AGE", str(age)) \
                      .replace("CONDITION", condition) \
                      .replace("X", str(days)) \
                      .replace("DID/DIDNOT", did_or_didnot) \
                      .replace("ADDENDUM", addendum)
    
    return summary

In [ ]:
file_path = r"C:\Users\edber\Desktop\Road to PhD\University of Washington\PPI\Palliative\2024.08.05 patients.xlsx"
df = pd.read_excel(file_path)

In [ ]:
df['model_name'] = ""
df['runtime_seconds'] = 0.0
df['question'] = ""
df['answer'] = ""

In [ ]:
for index, row in df.iterrows():
    question = generate_question(row, template)
    df.at[index, 'question'] = question
    query_result = query_store(client, question)
    
    df.at[index, 'model_name'] = query_result['model_name']
    df.at[index, 'runtime_seconds'] = query_result['runtime_seconds']
    df.at[index, 'question'] = query_result['question']
    df.at[index, 'answer'] = query_result['answer']

In [11]:
a = query_store(client, 
            "There was a female patient of age 75 who was hospitalized for Cancer for 7 days. During their hospitalization, they did discuss their goals of care. For each day, give me a summary detailing age, sex, and hospital course. The goals of care discussion took place on day 4.")

In [12]:
b = query_store(client, 
            "There was a female patient of age 75 who was hospitalized for Cancer for 7 days. During their hospitalization, they did not discuss their goals of care. For each day, give me a summary detailing age, sex, and hospital course.")

In [14]:
b

{'model_name': 'gpt-4-32k',
 'runtime_seconds': 15.441435813903809,
 'question': 'There was a female patient of age 75 who was hospitalized for Cancer for 7 days. During their hospitalization, they did not discuss their goals of care. For each day, give me a summary detailing age, sex, and hospital course.',
 'answer': "Day 1: A 75-year-old female patient was admitted to the hospital today for her cancer treatment. There was no discussion about her goals of care today. \n\nDay 2: On her second day, the 75-year-old female patient underwent some routine tests related to her cancer treatment. No goals of care were discussed today.\n\nDay 3: The patient, a 75-year-old woman, continued with her cancer treatment on the third day. Her goals of care were not discussed yet.\n\nDay 4: Today, on the fourth day of hospitalization, the 75-year-old female patient is stable and her treatment for cancer continues. Still, her goals of care weren't discussed.\n\nDay 5: On the fifth day, the patient, who